In [55]:
import pandas as pd

In [15]:
df = pd.read_csv("../data/all_2021_nfl_contracts.csv")

df["contract_length"] = pd.to_numeric(df.contract_length.replace("-", None))

df["contract_expires"] = df.contract_expires.replace("-", None)
df["contract_expires"] = pd.to_numeric(df.contract_expires)

df["contract_start"] = df.contract_expires - df.contract_length
df['_contract_amount'] = df.contract_amount.map('${:,.0f}'.format)


In [32]:
from bokeh.models import ColumnDataSource, LabelSet, Title, Axis
from bokeh.plotting import figure, show
from bokeh.sampledata.sprint import sprint
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
import bokeh.io
from bokeh.io import curdoc
from bokeh.themes import Theme
from bokeh.models import CustomJS, Dropdown


bokeh.io.reset_output()
bokeh.io.output_notebook()



Loading BokehJS ...

# Simple

In [1]:
f"{None}"

'None'

In [34]:
tooltips = [
    ("Player Name", "@player_name"),
    ("Position", "@position"),
    ("Experience (yrs)", "@exp"),
    ("Contract Amount", "@_contract_amount"),
    ("Guaranteed Amount", "@guaranteed_amount"),
    ("Contract Start", "@contract_start"),
    ("Contract End", "@contract_expires"),
    ("Acquired", "@acquired"),
]


p = figure(width=1500, 
           height=2000, title="Chicago Bears Contract Start and End Years Sorted by Contract Amount (including contract amts > 1 mil)", 
           x_range=(df.contract_start.min()-1, df.contract_expires.max()+1), y_range=df.player_name.tolist()[::-1],
           tooltips=tooltips,
          )


#OPTIONS = team_names
#multi_choice = MultiChoice(value=["foo", "baz"], options=OPTIONS)
#multi_choice.js_on_change("value", CustomJS(code="""console.log('multi_choice: value=' + this.value, this.toString())"""))
menu = [("Chicago Bears", "chicago-bears"), ("Arizona Cardinal", "arizona-cardinal")]

dropdown = Dropdown(label="Dropdown button", button_type="warning", menu=menu)
dropdown.js_on_event("menu_item_click", CustomJS(code="console.log('dropdown: ' + this.item, this.toString())"))


df = df[(df.contract_amount > 1000000) & (~df.contract_start.isnull()) & (df.tm.eq("chicago-bears"))]
source = ColumnDataSource(df.sort_values("contract_amount", ascending=True))
p.hbar(y="player_name", left='contract_start', right='contract_expires', height=0.4, source=source, fill_color="white", line_color="black")

p.title.text_font_size = "25px"
p.title.text_color ="#c83803"
p.title.align = 'center'
p.title.vertical_align = 'bottom'
p.min_border_top = 75
p.min_border_bottom = 75
p.yaxis.major_label_text_color = "#c83803"
p.xaxis.major_label_text_color = "#c83803"
p.background_fill_color = "#0B162A"
p.border_fill_color = "#0B162A"
p.ygrid.grid_line_alpha = 0.2
p.ygrid.grid_line_dash = [6, 4]
p.axis.axis_line_color = None
p.yaxis.major_label_text_font_size = "13pt"
p.xaxis.major_label_text_font_size = "13pt"
p.xaxis.axis_label="Contract Start and End Year"
p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.axis_label_text_color = "#c83803"

labels = LabelSet(x='contract_expires', y='player_name', text='_contract_amount', source=source,y_offset=-10, x_offset=2,render_mode='canvas', text_color="#c83803")
p.add_layout(labels)

show(p)

/home/jkonovsky/miniconda3/envs/myenv/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/home/jkonovsky/miniconda3/envs/myenv/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


# Added Filter